<a href="https://colab.research.google.com/github/quekhyg/NLP-Lyric-Generator/blob/main/%5Bword_level_%2B_whole_corpus%5D_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/SMU_MITB_NLP/NLP project/

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1MmY0pN1b5xL_C2CijM9ImcFM-UFt4bwr/NLP project


In [1]:
#import packages
import pandas as pd
import numpy as np
import os
import sys 
from keras.models import Sequential
from keras.layers import Activation, Flatten, Dropout, Dense, Embedding, TimeDistributed, GRU
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Data Preprocessing

In [3]:
path = "./songs_edit_revised/"
corpus = ''

all_files = os.listdir(path)
for file in all_files:
  with open(os.path.join(path, file)) as f:
    text = f.read()
    corpus+= '\n\n'+ text

In [4]:
components = corpus.split('\n\n')

In [5]:
print(len(components))

288


In [6]:
corpus_list = [string.split('\n') for string in components]
#corpus_list[:10]
for song in corpus_list:
  song.pop(0)

In [7]:
corpus_list[:10]

[[],
 ['Will you make this island',
  'Amazing in all ways',
  'Surprises every corner',
  'Delightful nights and days',
  'Will you take this country',
  'And turn it from a place',
  'To a home that greets you',
  'With smiles on every face',
  'Will you come on this brave journey',
  'Will you help to make it real',
  'Will you write us grand new stories',
  'Songs that everyone will feel'],
 ['So will you swim the current',
  'Will you scale new heights',
  'Will you make it happen',
  'Will you let your dreams take flight',
  'And will you make the difference',
  'Will you seize the day',
  'Will you live each moment',
  'Will you dare to find new ways'],
 ['Will you take this city',
  'And turn it from a place',
  'To a home that greets you',
  'With smiles on every face',
  'Will you come on this brave journey',
  'Will you help to make it real',
  'Will you write us grand new stories',
  'Songs that everyone will feel'],
 ['So will you swim the current',
  'Will you scale new h

In [8]:
df = pd.DataFrame({'song': corpus_list})

df['song'] = df['song'].str.join(', ')
pd.options.display.max_colwidth = 500
df=df.iloc[1:] 
df.head()

,song
1,"Will you make this island, Amazing in all ways, Surprises every corner, Delightful nights and days, Will you take this country, And turn it from a place, To a home that greets you, With smiles on every face, Will you come on this brave journey, Will you help to make it real, Will you write us grand new stories, Songs that everyone will feel"
2,"So will you swim the current, Will you scale new heights, Will you make it happen, Will you let your dreams take flight, And will you make the difference, Will you seize the day, Will you live each moment, Will you dare to find new ways"
3,"Will you take this city, And turn it from a place, To a home that greets you, With smiles on every face, Will you come on this brave journey, Will you help to make it real, Will you write us grand new stories, Songs that everyone will feel"
4,"So will you swim the current, Will you scale new heights, Will you make it happen, Will you let your dreams take flight, And will you make the difference, Will you seize the day, Will you live each moment, Will you dare to find new ways"
5,"Dare to find, Dare to find, Dare to find, New ways"


Generating lyrics for VERSE

In [20]:
#To run this cell if utilities.py is not in the same folder - this will upload utilities.py
# from google.colab import files
# src = list(files.upload().values())[0]
# open('utilities.py','wb').write(src)
# import utilities as U

In [11]:
import utilities as U

In [12]:
"""
Remove punctuations, decontractions, emojis and URLs
"""

df['song'] = df.apply(lambda row : U.remove_punct(row['song']), axis=1)
df['song'] = df.apply(lambda row : U.remove_url(row['song']), axis=1)
df['song'] = df.apply(lambda row : U.remove_emoji(row['song']), axis=1)
df['song'] = df.apply(lambda row : U.decontraction(row['song']), axis=1)
df['song']=  df.apply(lambda row : row['song'].lower(), axis=1)
df.head()

,song
1,will you make this island amazing in all ways surprises every corner delightful nights and days will you take this country and turn it from a place to a home that greets you with smiles on every face will you come on this brave journey will you help to make it real will you write us grand new stories songs that everyone will feel
2,so will you swim the current will you scale new heights will you make it happen will you let your dreams take flight and will you make the difference will you seize the day will you live each moment will you dare to find new ways
3,will you take this city and turn it from a place to a home that greets you with smiles on every face will you come on this brave journey will you help to make it real will you write us grand new stories songs that everyone will feel
4,so will you swim the current will you scale new heights will you make it happen will you let your dreams take flight and will you make the difference will you seize the day will you live each moment will you dare to find new ways
5,dare to find dare to find dare to find new ways


In [13]:
songs = df['song'].tolist()

In [14]:
songs[:10]

['will you make this island amazing in all ways surprises every corner delightful nights and days will you take this country and turn it from a place to a home that greets you with smiles on every face will you come on this brave journey will you help to make it real will you write us grand new stories songs that everyone will feel',
 'so will you swim the current will you scale new heights will you make it happen will you let your dreams take flight and will you make the difference will you seize the day will you live each moment will you dare to find new ways',
 'will you take this city and turn it from a place to a home that greets you with smiles on every face will you come on this brave journey will you help to make it real will you write us grand new stories songs that everyone will feel',
 'so will you swim the current will you scale new heights will you make it happen will you let your dreams take flight and will you make the difference will you seize the day will you live each

In [15]:
# Save verses in .txt file
with open('songs.txt', 'w',encoding="utf-8") as filehandle:  
    for listitem in songs:
        filehandle.write('%s\n' % listitem)

In [16]:
textFileName = 'songs.txt'
raw_text = open(textFileName, encoding = 'UTF-8').read()

Word-Level Lyrics Generation

In [27]:
"""
Generate song.
Reference: https://predictivehacks.com/word-based-text-generation/
"""
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers
import keras.utils as ku 

tokenizer = Tokenizer()

corpus = raw_text.lower().split("\n")
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1
 
# create input sequences using list of tokens
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)
 
 
# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
 
# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
 
label = ku.np_utils.to_categorical(label, num_classes=total_words)
 
 
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Dropout(0.2))
model.add(GRU(128))
model.add(Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 97, 100)           107000    
                                                                 
 dropout_2 (Dropout)         (None, 97, 100)           0         
                                                                 
 gru_3 (GRU)                 (None, 128)               88320     
                                                                 
 dense_2 (Dense)             (None, 535)               69015     
                                                                 
 dense_3 (Dense)             (None, 1070)              573520    
                                                                 
Total params: 837,855
Trainable params: 837,855
Non-trainable params: 0
_________________________________________________________________
None


In [18]:
'''
Neutral input seeds:
Whenever I think back 
And so this I know 
'''

seed_text = "whenever i think back"
next_words = 100

def generateLyrics(seed_text, model):
  next_words = 100   
  for _ in range(next_words):
      token_list = tokenizer.texts_to_sequences([seed_text])[0]
      token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
      predicted=np.argmax(model.predict(token_list, verbose=0) ,axis=1)

      output_word = ""
      for word, index in tokenizer.word_index.items():
          if index == predicted:
              output_word = word
              # break
      seed_text += " " + output_word
  return seed_text

lyrics_list=[]
lyrics = generateLyrics(seed_text, model)
lyrics_list.append(lyrics)
lyrics_list

['whenever i think back shining lamp lights play singaporeans goal unfurled put unity afar unity here oh goal began based sing gonna seems seems religion seems undivided try religion brings soar smile religion take place place weak id spring seems seems religion seems undivided religion brings soar take bright pick shining under blessing jewel ben based ours youve vast vast singaporeans aloud brings soar smile religion take place place weak adore such real harmony lions downs downs hours religion seems religion carry unity take bright stay firm id guess based youre shining story story seems told won won won won won won won won']

In [19]:
#Save Results to text file
outputdf=pd.DataFrame(lyrics_list)
outputdf.to_csv("whenever i think back_GRU.txt", header=False, 
                index=False, encoding="utf-8")

from google.colab import files
files.download("whenever i think back_GRU.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
seed_text = "and so this I know"
lyrics_list=[]
lyrics = generateLyrics(seed_text, model)
lyrics_list.append(lyrics)
lyrics_list

['and so this I know gonna here seems seems religion seems undivided religion brings soar take bright pick shining under blessing jewel ben based ours youve vast vast singaporeans aloud brings soar smile religion take place place weak adore such real harmony lions downs downs hours religion seems religion carry unity take bright stay firm id guess based youre shining story story seems told won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won']

In [22]:
#Save Results to text file
outputdf=pd.DataFrame(lyrics_list)
outputdf.to_csv("and so this I know_GRU.txt", header=False, index=False, encoding="utf-8")
files.download("and so this I know_GRU.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [23]:
'''
Negative input seeds:
I am tired of being what you want me to be 
Feeling so faithless, lost under the surface 
'''

seed_text = "i am tired of being what you want me to be"
lyrics_list=[]
lyrics = generateLyrics(seed_text, model)
lyrics_list.append(lyrics)
lyrics_list

['i am tired of being what you want me to be once singaporeans thousand lions town painted religion brings soar smile chains religion take place place undivided know gonna here seems seems seems religion seems undivided religion brings soar take bright pick shining under blessing jewel ben based ours youve vast vast singaporeans aloud brings soar smile religion take place place weak adore such real harmony lions downs downs hours religion seems religion carry unity take bright stay firm id guess based youre shining story story seems told won won won won won won won won won won won won won won won won won won won won won won won']

In [24]:
#Save Results to text file
outputdf=pd.DataFrame(lyrics_list)
outputdf.to_csv("i am tired of being what you want me to be_GRU.txt", header=False, index=False, encoding="utf-8")
files.download("i am tired of being what you want me to be_GRU.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [25]:
seed_text = "feeling so faithless lost under the surface"
lyrics_list=[]
lyrics = generateLyrics(seed_text, model)
lyrics_list.append(lyrics)
lyrics_list

['feeling so faithless lost under the surface listening listening religion seems religion brings soar take under blessing inspiration place place place weak adore across heat won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won won']

In [28]:
#Save Results to text file
outputdf=pd.DataFrame(lyrics_list)
outputdf.to_csv("feeling so faithless lost under the surface_GRU.txt", header=False, index=False, encoding="utf-8")
files.download("feeling so faithless lost under the surface_GRU.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [29]:
'''
Positive input seeds:
Relight our fire, we will find our way 
We will rise stronger together  
'''
seed_text = "relight our fire we will find our way"
lyrics_list=[]
lyrics = generateLyrics(seed_text, model)
lyrics_list.append(lyrics)
lyrics_list

['relight our fire we will find our way stood very stronger easy price fly price passing storys forever freedom finding understand streams nights about stood very minds wait minds breathe possible well im little strong nothings up very down higher gold chapter gold possible began across minds easy minds minds wait minds ill nights nights celebrate uk celebrations up freedom finding chance thankful thankful oh esplanade blessing across evening nights wall celebrate higher gold gold chapter gold amazing you chapter gold selfexplanatory freedom colleagues gold gold amazing down gold amazing gold down contribute neighbourhood cannot aheads signs despair bell you selfexplanatory used as gold amazing down gold amazing']

In [30]:
#Save Results to text file
outputdf=pd.DataFrame(lyrics_list)
outputdf.to_csv("relight our fire we will find our way_GRU.txt", header=False, index=False, encoding="utf-8")
files.download("relight our fire we will find our way_GRU.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [31]:
seed_text = "we will rise stronger together"
lyrics_list=[]
lyrics = generateLyrics(seed_text, model)
lyrics_list.append(lyrics)
lyrics_list

['we will rise stronger together easy lights cant nights uneventfullest uk play celebrations up freedom freedom finding finding chance sunny im im regardless im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im im']

In [32]:
#Save Results to text file
outputdf=pd.DataFrame(lyrics_list)
outputdf.to_csv("we will rise stronger together_GRU.txt", header=False, index=False, encoding="utf-8")
files.download("we will rise stronger together_GRU.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>